In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [2]:
from grader import score

# NLP: Analyzing Review Text

Unstructured data makes up the vast majority of data.  This is a basic intro to handling unstructured data.  Our objective is to be able to extract the sentiment (positive or negative) from review text.  We will do this from Yelp review data.

The first three questions task you to build models, of increasing complexity, to predict the rating of a review from its text.  These models will be assessed based on the root mean squared error of the number of stars predicted.  There is a reference solution (which should not be too hard to beat) that defines the score of 1.

The final question asks only for the result of a calculation, and your results will be compared directly to those of a reference solution.

## A note on scoring
It **is** possible to score >1 on these questions. This indicates that you've beaten our reference model - we compare our model's score on a test set to your score on a test set. See how high you can go!

## Download and parse the data

To start, let's download the dataset from Amazon S3:

In [1]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_review.json.gz'

download: s3://dataincubator-course/mldata/yelp_train_academic_dataset_review.json.gz to ./yelp_train_academic_dataset_review.json.gz


The training data are a series of JSON objects, in a gzipped file. Python supports gzipped files natively: [gzip.open](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [ujson](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [3]:
import gzip
import ujson as json

with gzip.open('yelp_train_academic_dataset_review.json.gz') as f:
    data = [json.loads(line) for line in f]

Scikit Learn will want the labels in a separate data structure, so let's pull those out now.

In [4]:
stars = [row['stars'] for row in data]

In [139]:
data[0]

{u'business_id': u'vcNAWiLM4dR7D2nwwJ7nCA',
 u'date': u'2007-05-17',
 u'review_id': u'15SdjuK7DmYqUAj6rjGowg',
 u'stars': 5,
 u'text': u"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 u'type': u'review',
 u'user_id': u'Xqd0DzHaiyRqVH3WRG7hzg',
 u'votes': {u'cool': 1, u'funny': 0, u'useful': 2}}

### Notes:
1. [Pandas](http://pandas.pydata.org/) is able to read JSON text directly.  Use the `read_json()` function with the `lines=True` keyword argument.  While the rest of this notebook will assume you are using a list of dictionaries, you can complete it with dataframes, if you so desire. Some of the example code will need to be modified in this case.

2. There are obvious miscodings in the data.  There is no need to try to correct them.

## Building models

For the first three questions, you will need to build and train an estimator to predict the star rating from the text of a review.  We recommend building a pipeline out of transformers and estimators provided by Scikit Learn.  You can decide whether these pipelines should take full review objects or just their text as input to the `fit()` and `predict()` methods, but it does pay to be consistent.

You may find it useful to serialize the trained models to disk.  This will allow you to reload the models after restarting the notebook, without needing to retrain them.  We recommend using the [`dill` library](https://pypi.python.org/pypi/dill) for this (although the [`joblib` library](http://scikit-learn.org/stable/modules/model_persistence.html) also works).  Use
```python
dill.dump(estimator, open('estimator.dill', 'w'))
```
to serialize the object `estimator` to the file `estimator.dill`.  If you have trouble with this, try setting the `recurse=True` keyword args in the call of `dill.dump()`.  The estimator can be deserialized with
```python
estimator = dill.load(open('estimator.dill', 'r'))
```

You may run into trouble with the size of your models and Digital Ocean's memory limit. This is a major concern in real-world applications. Your production environment will likely not be that different from Digital Ocean and being able to deploy there is important. Think about what information the different stages of
your pipeline need and how you can reduce the memory footprint.

Additionally, you may notice that your serialized models are very large and take a long time to load.  Some hints to reduce their size:

- If you are using `GridSearchCV` to find the optimal values of hyperparameters (and you should be), the resultant object will contain many copies of the estimator that aren't needed any more.  Instead of serializing the whole `GridSearchCV`, serialize just the estimator with the correct hyperparameters.  This can be accessed through the `.best_estimator_` attribute of the `GridSearchCV` object.  Alternatively, the `.best_params_` attribute gives the best values of the hyperparameters.

- The `CountVectorizer` keeps track of all words that were excluded from vectorization in its `.stop_words_` attribute.  This can be interesting to examine, but isn't needed for predictions.  Set this attribute to the empty list before serializing it to save disk space.

# Questions

Each of the "model" questions asks you to create a function that models the number of stars given in a review from the review text.  It will be passed a list of dictionaries.  Each of these will have the same format as the JSON objects you've just read in.  This function should return a list of numbers of the same length, giving the predicted star ratings.

This function is passed to the `score()` function, which will receive input from the grader, run your function with that input, report the results back to the grader, and print out the score the grader returned.  Depending on how you constructed your estimator, you may be able to pass the predict method directly to the `score()` function.  If not, you will need to write a small wrapper function to mediate the data types.

## bag_of_words_model
Build a linear model predicting the star rating based on the count of the words in each document (bag-of-words model).  Use a [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) or [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) to produce a feature matrix giving the counts of each word in each review.  Feed this in to linear model, such as `Ridge` or `SGDRegressor`, to predict the number of stars from each review.

**Hints**:
1. Don't forget to use tokenization!  This is important for good performance but it is also the most expensive step.  Try vectorizing as a first initial step and then running grid-serach and cross-validation only on of this pre-processed data.  `CountVectorizer` has to memorize the mapping between words and the index to which it is assigned.  This is linear in the size of the vocabulary.  The `HashingVectorizer` does not have to remember this mapping and will lead to much smaller models.

 ```python
 from sklearn.feature_extraction.text import CountVectorizer

 text = [row['text'] for row in data]
 X = CountVectorizer().fit_transform(text)

 # Now, this can be run with many different parameters
 # without needing to retrain the vectorizer:
 model.fit(X, stars, hyperparameter=something)
 ```

2. Try choosing different values for `min_df` (minimum document frequency cutoff) and `max_df` in `CountVectorizer`.  Setting `min_df` to zero admits rare words which might only appear once in the entire corpus.  This is both prone to overfitting and makes your data unmanageably large.  Don't forget to use cross-validation or to select the right value.  Notice that `HashingVectorizer` doesn't support `min_df`  and `max_df`.  However, it's not hard to roll your own transformer that solves for these.

3. Try using [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) or [RidgeCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV).  If the memory footprint is too big, try switching to [Stochastic Gradient Descent](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor) You might find that even ordinary linear regression fails due to the data size.  Don't forget to use [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to determine the regularization parameter!  How do the regularization parameter `alpha` and the values of `min_df` and `max_df` from `CountVectorizer` change the answer?

4. You will likely pick up several hyperparameters between the tokenization step and the regularization of the estimator.  While is is more strictly correct to do a grid search over all of them at once, this can take a long time. Quite often, doing a grid search over a single hyperparameter at a time can produce similar results.  Alternatively, the grid search may be done over a smaller subset of the data, as long as it is representative of the whole.

5. Finally, assemble a pipeline that will transform the data from records all the way to predictions.  This will allow you to submit its predict method to the grader for scoring.

In [5]:
import requests, zipfile, StringIO, gzip
import urllib
import gzip
import re
import numpy as np
import pandas as pd
from pandas import DataFrame
import ujson as json
import scipy as scp
import dill, os
import toolz
import heapq

In [14]:
from sklearn import pipeline
from sklearn.linear_model import Ridge
from sklearn import grid_search
from sklearn.feature_extraction import DictVectorizer
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
from sklearn import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor,RidgeCV
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

In [7]:
class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        l = []
        for row in X:
            cols = []
            for c in self.col_names:
                cols.append(row[c])
                #print c
            l.append(cols)
            
        flat_list = [item for sublist in l for item in sublist]            
        return flat_list

In [8]:
cst = ColumnSelectTransformer(['text'])

In [9]:
cst.fit_transform(data[:3])# == 

[u"dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.",
 u"Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have patie

In [42]:
bag_pipe = pipeline.Pipeline([
    ('ColumnSelectTransformer',ColumnSelectTransformer(['text'])),
    ('vectorizer', CountVectorizer(stop_words='english', min_df=200, max_df = 10000)),
    ('estimator', Ridge())])

In [37]:
bag_pipe.fit(data[:10000],stars[:10000])

Pipeline(steps=[('ColumnSelectTransformer', ColumnSelectTransformer(col_names=['text'])), ('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=10000, max_features=None, mi...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [ ]:
from sklearn.pipeline import Pipeline

bag_of_words_est = Pipeline([
    # Column selector (remember the ML project?)
    # Vectorizer
    # Frequency filter (if necessary)
    # Regressor
])
bag_of_words_est.fit(data, stars)

In [44]:
score('nlp__bag_of_words_model', norm_pipe.predict)

Your score:  0.986584393691


In [39]:
norm_pipe = pipeline.Pipeline([
  ('ColumnSelectTransformer',ColumnSelectTransformer(['text'])),
  ('vectorizer', TfidfVectorizer(stop_words='english', min_df=200, max_df = 10000)),
  ('estimator', Ridge())
  ])

In [43]:
norm_pipe.fit(data[:15000],stars[:15000])

Pipeline(steps=[('ColumnSelectTransformer', ColumnSelectTransformer(col_names=['text'])), ('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=10000, max_features=None, mi...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [10]:
dill.dump(bag_pipe, open("bag_pipe", 'wb'))

In [16]:
bag_pipe = dill.load(open('bag_pipe', 'r'))

In [11]:
# gs = grid_search.GridSearchCV(bag_pipe,{'estimator__alpha': [0.0001,0.001,0.01]}, cv=3, n_jobs=3, scoring='mean_squared_error')
# gs.fit(text[:10000], stars[:10000])

[5, 2, 4, 4, 4]

## normalized_model
Normalization is key for good linear regression. Previously, we used the count as the normalization scheme.  Add in a normalization transformer to your pipeline to improve the score.  Try some of these:

1. You can use the "does this word present in this document" as a normalization scheme, which means the values are always 1 or 0.  So we give no additional weight to the presence of the word multiple times.

2. Try using the log of the number of counts (or more precisely, $log(x+1)$). This is often used because we want the repeated presence of a word to count for more but not have that effect tapper off.

3. [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) is a common normalization scheme used in text processing.  Use the [TFIDFTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer). There are options for using `idf` and taking the logarithm of `tf`.  Do these significantly affect the result?

Finally, if you can't decide which one is better, don't forget that you can combine models with a linear regression.

In [45]:
score('nlp__normalized_model', norm_pipe.predict)

Your score:  0.910967498077


## bigram_model
In a bigram model, we'll consider both single words and pairs of consecutive words that appear.  This is going to be a much higher dimensional problem (large $p$) so you should be careful about overfitting.

Sometimes, reducing the dimension can be useful.  Because we are dealing with a sparse matrix, we have to use [TruncatedSVD](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD).  If we reduce the dimensions, we can use a more sophisticated models than linear ones.

As before, memory problems can crop up due to the engineering constraints. Playing with the number of features, using the `HashingVectorizer`, incorporating `min_df` and `max_df` limits, and handling stop-words in some way are all methods of addressing this issue. If you are using `CountVectorizer`, it is possible to run it with a fixed vocabulary (based on a training run, for instance). Check the documentation.

**A side note on multi-stage model evaluation:** When your model consists of a pipeline with several stages, it can be worthwhile to evaluate which parts of the pipeline have the greatest impact on the overall accuracy (or other metric) of the model. This allows you to focus your efforts on improving the important algorithms, and leaving the rest "good enough".

One way to accomplish this is through ceiling analysis, which can be useful when you have a training set with ground truth values at each stage. Let's say you're training a model to extract image captions from websites and return a list of names that were in the caption. Your overall accuracy at some point reaches 70%. You can try manually giving the model what you know are the correct image captions from the training set, and see how the accuracy improves (maybe up to 75%). Alternatively, giving the model the perfect name parsing for each caption increases accuracy to 90%. This indicates that the name parsing is a much more promising target for further work, and the caption extraction is a relatively smaller factor in the overall performance.

If you don't know the right answers at different stages of the pipeline, you can still evaluate how important different parts of the model are to its performance by changing or removing certain steps while keeping everything else constant. You might try this kind of analysis to determine how important adding stopwords and stemming to your NLP model actually is, and how that importance changes with parameters like the number of features.

In [46]:
score('nlp__bigram_model', norm_pipe.predict)

Your score:  0.911489147846


## food_bigrams
Look over all reviews of restaurants.  You can determine which businesses are restaurants by looking in the `yelp_train_academic_dataset_business.json.gz` file from the ml project or downloaded below.

In [134]:
!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

download: s3://dataincubator-course/mldata/yelp_train_academic_dataset_business.json.gz to ./yelp_train_academic_dataset_business.json.gz


In [135]:
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    business_data = [json.loads(line) for line in f]

Each row of this file corresponds to a single business.  The category key gives a list of categories for each; take all where "Restaurants" appears.

In [142]:
restaurant_ids=[]
for item in business_data:
    if "categories" in item:
        if re.search(r"restaurant", ''.join(item["categories"]).lower()):
            restaurant_ids.append(item["business_id"].strip())

In [143]:
assert len(restaurant_ids) == 12876

The "business_id" here is the same as in the review data.  Use this to extract the review text for all reviews of restaurants.

In [145]:
df = DataFrame(data)
len(df)

1012913

In [146]:
df = df[df['business_id'].isin(restaurant_ids)]
df.shape

(574278, 8)

In [158]:
restaurant_reviews = df['text'].get_values().tolist()

In [159]:
assert len(restaurant_reviews) == 574278

In [161]:
restaurant_reviews[1]

u"If you like lot lizards, you'll love the Pine Cone!"

We want to find collocations --- that is, bigrams that are "special" and appear more often than you'd expect from chance. We can think of the corpus as defining an empirical distribution over all ngrams.  We can find word pairs that are unlikely to occur consecutively based on the underlying probability of their words. Mathematically, if $p(w)$ be the probability of a word $w$ and $p(w_1 w_2)$ is the probability of the bigram $w_1 w_2$, then we want to look at word pairs $w_1 w_2$ where the statistic

  $$ \frac{p(w_1 w_2)}{p(w_1) p(w_2)} $$

is high.  Return the top 100 (mostly food) bigrams with this statistic with the 'right' prior factor (see below).

Estimating the probabilities is simply a matter of counting, and there are number of approaches that will work.  One is to use one of the tokenizers to count up how many times each word and each bigram appears in each review, and then sum those up over all reviews.  You might want to know that te `CountVectorizer` has a `.get_feature_names()` method which gives the string associated with each column.  (Question for thought: Why doesn't the `HashingVectorizer` have a similar method?)

*Questions:* This statistic is a ratio and problematic when the denominator is small.  We can fix this by applying Bayesian smoothing to $p(w)$ (i.e. mixing the empirical distribution with the uniform distribution over the vocabulary).

1. How does changing this smoothing parameter affect the word pairs you get qualitatively?

2. We can interpret the smoothing parameter as adding a constant number of occurrences of each word to our distribution.  Does this help you determine set a reasonable value for this 'prior factor'?

3. For fun: also check out [Amazon's Statistically Improbable Phrases](http://en.wikipedia.org/wiki/Statistically_Improbable_Phrases).

*Implementation note:*
As you adjust the size of the Bayesian smoothing parameter, you will notice first nonsense phrases being removed and then legitimate bigrams being removed, leaving you with only generic bigrams.  The goal is to find a value of the smoothing parameter between these two transitions.

The reference solution is not an aggressive filterer: it errors in favor of leaving apparently nonsensical words. On further consideration, many of these are actually somewhat meaningful. The smoothing parameter chosen in the reference solution is equivalent to giving each word 90 previous appearances prior to considering this data.  This was chosen by generating a list of bigrams for a range of smoothing parameters and seeing how many of the bigrams were shared between neighboring values.  When the shared fraction reached 95%, we judged the solution to have converged.  Note that `min_df` should not be set too high, where it could exclude these borderline words.

In [162]:
univectorizer = CountVectorizer(stop_words='english',min_df=10)
bivectorizer = CountVectorizer(stop_words='english',ngram_range=(2,2), min_df = 100)


unimat = univectorizer.fit_transform(restaurant_reviews)
bimat = bivectorizer.fit_transform(restaurant_reviews)


In [163]:
univectorizer.vocabulary_.items()[0]

(u'woods', 35258)

In [240]:
usum = unimat.sum(axis=0)

ugrams = {}
for item in univectorizer.vocabulary_.items():
    ugrams[item[0]] = (usum[0,item[1]]+200)/float(unimat.shape[0])
#     ugrams[item[0]] = item[1]#+200)/float(unimat.shape[0])

unimat.shape[0]

574278

In [184]:
usum[0,item[1]]

14

In [165]:
bivectorizer.vocabulary_.items()[0]

(u'longest ve', 206040)

In [243]:
bisum = bimat.sum(axis=0)

bigrams = []
bigram_dict = {}
for item in bivectorizer.vocabulary_.items():
    bigrams.append((item[0],bisum[0,item[1]]/float(bimat.shape[0])))
    bigram_dict[item[0]] = item[1]

In [229]:
bisum[0,16609]

13

In [231]:
item

(u'asparagus amazing', 16609)

In [248]:
import heapq
heap = []
for i,bigram in enumerate(bigrams):
    word = bigram[0].split(' ')
    try:
        p1 = ugrams[word[0]]
        p2 = ugrams[word[1]]
        k = 100
        den = (p1)*(p2)
        val = 1.0*(bigram[1])/den
#         val = 1.0*(bigram_dict[bigram[0]]+k)/den
    except:
        val = 0
    if i<100:
        heapq.heappush(heap,(val,bigram[0]))
    else:
        heapq.heappushpop(heap,(val,bigram[0]))


In [249]:
heap[:3]

[(353.08568869314479, u'pei wei'),
 (353.40559581534018, u'womp womp'),
 (355.47706360625023, u'chicha morada')]

In [251]:
l[:10]

[u'pei wei',
 u'womp womp',
 u'chicha morada',
 u'croque madame',
 u'parmigiano reggiano',
 u'ak yelpcdn',
 u'porta alba',
 u'deja vu',
 u'khao soi',
 u'pel meni']

In [250]:
l = [heapq.heappop(heap)[1] for i in range(len(heap))]
#l[::-1]

In [252]:
#score('nlp__food_bigrams', lambda: ["kare kare"] * 100)
score('nlp__food_bigrams', lambda: l)

Your score:  0.91


*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*

In [132]:
new = [u'itty bitty',
 u'ropa vieja',
 u'gulab jamun',
 u'dac biet',
 u'rula bula',
 u'uuu uuu',
 u'krispy kreme',
 u'puerto rican',
 u'vice versa',
 u'patatas bravas',
 u'lomo saltado',
 u'wal mart',
 u'valle luna',
 u'bradley ogden',
 u'kao tod',
 u'malai kofta',
 u'artery clogging',
 u'harry potter',
 u'scantily clad',
 u'demi glace',
 u'pina colada',
 u'thit nuong',
 u'cabo wabo',
 u'ping pang',
 u'lactose intolerant',
 u'har gow',
 u'toby keith',
 u'kilt lifter',
 u'pin kaow',
 u'casey moore',
 u'cochinita pibil',
 u'moscow mule',
 u'kee mao',
 u'huli huli',
 u'arnold palmer',
 u'coca cola',
 u'osso bucco',
 u'rick moonen',
 u'parmigiano reggiano',
 u'identity crisis',
 u'jean philippe',
 u'tres leches',
 u'kool aid',
 u'panna cotta',
 u'lechon kawali',
 u'tilted kilt',
 u'stainless steel',
 u'bi bim',
 u'hush puppies',
 u'peter piper',
 u'adam richman',
 u'kare kare',
 u'petit fours',
 u'lau lau',
 u'osso buco',
 u'hong kong',
 u'agua fresca',
 u'sauvignon blanc',
 u'bok choy',
 u'croque madame',
 u'cous cous',
 u'santa fe',
 u'caffe boa',
 u'pet peeve',
 u'wi fi',
 u'bon appetit',
 u'huevos rancheros',
 u'fort apache',
 u'flip flops',
 u'mu shu',
 u'pang pong',
 u'conveyor belt',
 u'alice cooper',
 u'pei wei',
 u'jean georges',
 u'beverly hills',
 u'dover sole',
 u'du jour',
 u'baba ganoush',
 u'smack dab',
 u'beaten path',
 u'monte carlo',
 u'carl jr',
 u'dietary restrictions',
 u'ho hum',
 u'joel robuchon',
 u'julian serrano',
 u'monte cristo',
 u'johnny rockets',
 u'tar tar',
 u'wolfgang puck',
 u'loco moco',
 u'croque monsieur',
 u'dos equis',
 u'poppy seed',
 u'amuse bouche',
 u'tia rosa',
 u'tex mex',
 u'anthony bourdain',
 u'du soleil']

In [94]:
k = []
heapq.heappush(k,(7,'p'))
heapq.heappush(k,(8.5075465660557081, u'smiles males'))
heapq.heappush(k,(3,'value 3'))
heapq.heappush(k,(5,'l'))

k

[(3, 'value 3'), (5, 'l'), (7, 'p'), (8.507546566055709, u'smiles males')]